<a href="https://colab.research.google.com/github/AliciaFalconCaro/DABesselMotorDetection/blob/main/DAOT_QuadeAndNemenyiTests.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install scikit-posthocs


In [ ]:
import numpy as np
from scipy.io import loadmat

#DAOT ideal vs Competing Methods:
mat_data = loadmat('Dataset1_SubGestures2-3Bessels_LOSOkfolds.mat')
IdealDAOTModel = np.array(mat_data['ValAccuracy'])/100
print(IdealDAOTModel.shape)

#Competing Method 1: 2/3 DAOT
mat_data = loadmat('Ablation_NO_DA_Dataset1_SubGestures2-3Bessels_LOSOkfolds.mat')
DAOTModel2_3 = np.array(mat_data['ValAccuracy'])/100

#Competing Method 1: EEGNet
mat_data = loadmat('Ablation_NO_GC_Dataset1_SubGestures2-3Bessels_LOSOkfolds.mat')
EEGNetModel = np.array(mat_data['ValAccuracy'])/100
print(EEGNetModel.shape)

#Competing Method 1: SCCNet
mat_data = loadmat('Ablation_ReducedCNN_1Conv2D_Dataset1_SubGestures2-3Bessels_LOSOkfolds.mat')
SCCNetModel = np.array(mat_data['ValAccuracy'])/100

#Competing Method 1: SCCNet
mat_data = loadmat('Ablation_ReducedCNN_2Filters_Dataset1_SubGestures2-3Bessels_LOSOkfolds.mat')
SCCNetModelReduced = np.array(mat_data['ValAccuracy'])/100

/usr/local/lib/python3.10/dist-packages/scipy/io/matlab/_mio.py:227: MatReadWarning: Duplicate variable name "None" in stream - replacing previous with new
Consider mio5.varmats_from_mat to split file into single variable files
  matfile_dict = MR.get_variables(variable_names)


(14, 1)
(14, 1)


In [ ]:
# Combine the arrays into a single (14, 4) array
combined_data = np.column_stack((IdealDAOTModel, DAOTModel2_3, EEGNetModel, SCCNetModel,SCCNetModelReduced))

print("Combined Data Shape:", combined_data.shape)  # Should output (14, 4)
print("Combined Data:\n", combined_data)

Combined Data Shape: (14, 5)
Combined Data:
 [[0.7        0.1        0.8        0.78571426 0.5142857 ]
 [0.6        0.3        0.8        0.62857143 0.7       ]
 [0.8        0.2        0.9        0.7        0.8285714 ]
 [0.6142857  0.2        0.7        0.91428574 0.7714286 ]
 [0.8285714  0.         0.6        0.84285713 0.6       ]
 [0.64285713 0.         0.6        0.6        0.8       ]
 [0.8        0.3        0.7        0.7        0.84285713]
 [0.62857143 0.2        0.6        0.7        0.9       ]
 [0.6        0.1        0.6        0.8        0.5857143 ]
 [0.68571426 0.2        0.5        0.7        0.8       ]
 [0.7        0.         0.7        0.7        0.94285713]
 [0.8        0.2        0.9        0.5        0.35714287]
 [0.52857143 0.1        0.8        0.68571426 0.6       ]
 [0.81428574 0.         0.7        0.68571426 0.75714287]]


In [ ]:
import pandas as pd
from scipy.stats import friedmanchisquare
import scikit_posthocs as sp

# Step 1: Rank the models within each fold
ranked_data = np.apply_along_axis(lambda x: pd.Series(x).rank().values, axis=1, arr=combined_data)

# Step 2: Calculate fold weights based on the differences of model ranks
fold_means = np.mean(ranked_data, axis=1)
fold_weights = np.abs(ranked_data - fold_means[:, None]).sum(axis=1)

# Step 3: Calculate weighted ranks
weighted_ranks = ranked_data * fold_weights[:, None]

# Step 4: Compute Quade's test statistic
# Average the weighted ranks for each model
model_means = np.mean(weighted_ranks, axis=0)

# Compute the mean of weighted ranks overall
overall_mean = np.mean(weighted_ranks)

# Quade's F-statistic formula
ss_between = len(combined_data) * np.sum((model_means - overall_mean) ** 2)
ss_within = np.sum((weighted_ranks - model_means) ** 2)
quade_statistic = (ss_between / (combined_data.shape[1] - 1)) / (ss_within / (combined_data.size - combined_data.shape[1]))

print("Quade Statistic:", quade_statistic)

# Step 5: Post-hoc tests if significant (pairwise comparisons)
# Use Nemenyi test to find out which models differ if the Quade statistic is significant
p_values = sp.posthoc_nemenyi_friedman(ranked_data)
print("Post-hoc p-values (Nemenyi test):\n", p_values)

Quade Statistic: 16.27274185083678
Post-hoc p-values (Nemenyi test):
           0         1         2         3         4
0  1.000000  0.001564  0.993602  0.993602  0.919248
1  0.001564  1.000000  0.000278  0.000278  0.000041
2  0.993602  0.000278  1.000000  1.000000  0.993602
3  0.993602  0.000278  1.000000  1.000000  0.993602
4  0.919248  0.000041  0.993602  0.993602  1.000000


For the ablation study, we compare 4 ablated